In [16]:
from pathlib import Path
import os
from IPython import get_ipython
from IPython.core.magic import register_cell_magic

ipython = get_ipython()


@register_cell_magic
def pybash(line, cell):
    ipython.run_cell_magic("bash", "", cell.format(**globals()))

In [17]:
tmp_dir = Path("../tmp/3ftx_alignment")
data_dir = Path("../data/3Ftx")

cobra_file = data_dir / "cobra.cif"
krait_file = data_dir / "krait.cif"
snake_file = data_dir / "snake.cif"


os.makedirs(tmp_dir, exist_ok=True)

## Commandline Usage


In [18]:
cobra_path = str(cobra_file)
cobra_matrix = str(tmp_dir / "cobra_matrix.npy")
cobra_info = str(tmp_dir / "cobra_info.json")
cobra_out = str(tmp_dir / "cobra.svg")

krait_path = str(krait_file)
krait_matrix = str(tmp_dir / "krait_matrix.npy")
krait_info = str(tmp_dir / "krait_info.json")
krait_out = str(tmp_dir / "krait.svg")

snake_path = str(snake_file)
snake_matrix = str(tmp_dir / "snake_matrix.npy")
snake_info = str(tmp_dir / "snake_info.json")
snake_out = str(tmp_dir / "snake.svg")

db_path = str(Path("../out/alignment_db_test"))
min_p = 0.001


In [19]:
%%pybash
# Access Python variables in bash using curly braces

uv run flatprot align {cobra_path} {cobra_matrix} {cobra_info} -d {db_path} --min-probability {min_p} --quiet
uv run flatprot align {krait_path} {krait_matrix} {krait_info} -d {db_path} --min-probability {min_p} --quiet
# uv run flatprot align {snake_path} {snake_matrix} {snake_info} -d {db_path} --min-probability {min_p} --quiet


In [20]:
%%pybash
uv run flatprot project {cobra_path} -o {cobra_out} --matrix {cobra_matrix} --quiet
uv run flatprot project {krait_path} -o {krait_out} --matrix {krait_matrix} --quiet
# uv run flatprot project {snake_path} -o {snake_out} --matrix {snake_matrix} --quiet


In [21]:
# Display the three generated SVGs next to each other
from IPython.display import display, HTML


def display_svg_files(svg_files, titles=None, width="30%"):
    """
    Display multiple SVG files side by side.

    Args:
        svg_files: List of paths to SVG files
        titles: Optional list of titles for each SVG
        width: Width of each SVG (as CSS value)
    """
    if titles is None:
        titles = [f"SVG {i + 1}" for i in range(len(svg_files))]

    html = '<div style="display: flex; justify-content: space-between;">'

    for i, (svg_file, title) in enumerate(zip(svg_files, titles)):
        with open(svg_file, "r") as f:
            svg_content = f.read()

        # Modify SVG to constrain its width
        svg_content = svg_content.replace(
            "<svg ", f'<svg style="width:100%; height:auto;" '
        )

        html += f"""
        <div style="width: {width}; border: 1px solid black; text-align: center; border-radius: 20% 0 20% 0">
            <h3>{title}</h3>
            {svg_content}
        </div>
        """

    html += "</div>"
    display(HTML(html))


# Display the three toxin structures side by side
display_svg_files(
    [
        cobra_out,
        krait_out,
    ],
    titles=["Cobra", "Krait", "Snake"],
)